In [2]:
from src.scripts.util.common import *
import pandas as pd

columns = ["# of documents", "#of categories", "category distribution", "avg # of words per document", "avg # of NE per document", "feature # per representation"]
df = pd.DataFrame(columns = columns, index=[d.value for d in Dataset])
TARGET_COL = 'target'
TEXT_COL = 'text'

In [3]:
for d in Dataset:
    train = load_raw(d, "train")
    test = load_raw(d, "test")

    df.loc[d.value, "# of documents"] = train.shape[0]+test.shape[0]

    categories = train[TARGET_COL].unique().tolist()
    df.loc[d.value,"#of categories"] = f"{len(categories)}:{','.join([str(c) for c in categories])}"

    counts_train =train[TARGET_COL].value_counts()
    counts_test = test[TARGET_COL].value_counts()
    df.loc[d.value,"category distribution"] = ', '.join([f"{c}: {counts_test[c] + counts_train[c]}" for c in categories])

    f_per_state = ""
    for s in [State.BIO, State.STD, State.DOUBLE]:
        test_state = read_as_dataframe(d,s,"test")
        f_per_state += f"{s.value.upper()}: {test_state.shape[1]} "
    df.loc[d.value,"feature # per representation"] = f_per_state



In [4]:
from src.representations import SpacyNEClassifier
tagged = {}
# "avg # of words per document"
for d in Dataset:
    train = load_raw(d, "train")
    test = load_raw(d, "test")
    whole = pd.concat([train,test])
    print(whole.shape)
    ner = SpacyNEClassifier()
    tagged[d] = ner.predict(whole[TEXT_COL])

(16800, 2)
(10876, 2)
(2225, 2)
(10000, 2)
(10000, 2)


KeyboardInterrupt: 

In [5]:
df

,# of documents,#of categories,category distribution,avg # of words per document,avg # of NE per document,feature # per representation
ag-news,16800,"4:1,2,3,4","1: 4200, 2: 4200, 3: 4200, 4: 4200",NaN,NaN,BIO: 7669 STD: 5930 DOUBLE: 8362
disasters,10876,"3:Relevant,Not Relevant,Can't Decide","Relevant: 4673, Not Relevant: 6187, Can't Deci...",NaN,NaN,BIO: 2761 STD: 2250 DOUBLE: 2854
bbc,2225,"5:tech,politics,sport,entertainment,business","tech: 401, politics: 417, sport: 511, entertai...",NaN,NaN,BIO: 7670 STD: 5365 DOUBLE: 8104
fine-foods,10000,"5:5,2,4,1,3","5: 6183, 2: 590, 4: 1433, 1: 932, 3: 862",NaN,NaN,BIO: 4461 STD: 3356 DOUBLE: 4792
imdb,10000,"2:positive,negative","positive: 5028, negative: 4972",NaN,NaN,BIO: 13526 STD: 9738 DOUBLE: 14251
